In [1]:
###########
# Imports #
###########
import random
from dorna2 import Dorna

In [1]:
###############
# Config Cell #
###############

robot_ip_address = "localhost"  # robot ip address
tcp_output = [0, 0, 1]  # output pin for the suction cup(pin, off state, on state)
tcp_frame = [0, 0, 20, 0, 0, 0]  # define the tool length and position
speed = 0.5  # speed of the robot

sleep_period = 1 # period allowing other robot to react to see if pin 5=0 and to set its pin 5=0

playing_flag = [5, 0, 1]  # [output pin, not playing state, playing state]
chip_place_flag = [1, 2, 3, 4]  # io pin we use to signal placment position

chip_pick_position = [[x, y, z, a, b, c],  # position to grab first chip
                  [x, y, z, a, b, c],  # position to grab second chip
                  [x, y, z, a, b, c],  # position to grab third chip
                  [x, y, z, a, b, c],  # position to grab fourth chip
                  [x, y, z, a, b, c]]  # position to grab fifth chip

chip_place_position = [[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],  # Top row
                    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], # middle row 
                    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]] # bottom row

# setting the placement positions on the board to corresponding to pin values
chip_place = {
    (0, 0): [0, 0, 0, 0],  # Top-left
    (0, 1): [0, 0, 0, 1],  # Top-middle
    (0, 2): [0, 0, 1, 0],  # Top-right
    (1, 0): [0, 1, 0, 0],  # Middle-left
    (1, 1): [0, 1, 0, 1],  # Center
    (1, 2): [0, 1, 1, 0],  # Middle-right
    (2, 0): [1, 0, 0, 0],  # Bottom-left
    (2, 1): [1, 0, 0, 1],  # Bottom-middle
    (2, 2): [1, 0, 1, 0]   # Bottom-right
}

middle_pose = [0, 0, 0, 0, 0, 0]# here is the safe postion for robot between picking and placing
safe_pose = [1, 1, 1, 1 ,1 ,1 ]# safe position so other robot can work

NameError: name 'x' is not defined

In [ ]:
####################
# Helper Functions #
####################
def check_win(board, player):  # checks for a win
    win_conditions = [
        # Check rows
        [board[0][0], board[0][1], board[0][2]],
        [board[1][0], board[1][1], board[1][2]],
        [board[2][0], board[2][1], board[2][2]],
        # Check columns
        [board[0][0], board[1][0], board[2][0]],
        [board[0][1], board[1][1], board[2][1]],
        [board[0][2], board[1][2], board[2][2]],
        # Check diagonals
        [board[0][0], board[1][1], board[2][2]],
        [board[2][0], board[1][1], board[0][2]]
    ]
    return [player, player, player] in win_conditions # checks if a win condition is met


def chip_postion(board):  # finds an open position on the board
    # remaining positions
    remaining_position = [[i, j] for i in range(3) for j in range(3) if matrix[i][j] is None]
    
    # reshuffle to make it random
    random.shuffle(remaining_position)

    # return the first element
    return remaining_position[0]

In [ ]:
robot = Dorna() # create the Dorna object

# tic-tac-toe board matrix
board = [[None, None, None],
         [None, None, None],
         [None, None, None]]

# setting the placement positions on the board to corresponding to pin values
# (row, col): (pin 0, pin 1, pin 2, pin 3)
position_to_pins = {
    (0, 0): [0, 0, 0, 0],  # Top-left
    (0, 1): [0, 0, 0, 1],  # Top-middle
    (0, 2): [0, 0, 1, 0],  # Top-right
    (1, 0): [0, 1, 0, 0],  # Middle-left
    (1, 1): [0, 1, 0, 1],  # Center
    (1, 2): [0, 1, 1, 0],  # Middle-right
    (2, 0): [1, 0, 0, 0],  # Bottom-left
    (2, 1): [1, 0, 0, 1],  # Bottom-middle
    (2, 2): [1, 0, 1, 0]   # Bottom-right
}

In [ ]:
robot.connect(robot_ip_address)  # connect to robot
robot.set_motor(1)  # turn on the robot motors
robot.jmove(safe_pose)  #set safe initial position
robot.set_output(tcp_output[0], tcp_output[1]) # set output, (pin, state)
robot.set_output(playing_flag[0], playing_flag[1]) # set output 

In [ ]:
# game loop #
pick_index = 0  # start with the first grab position

for move_number in range(9):  # max 9 moves in a Tic Tac Toe game

    robot.wait(**{"in"+str(playing_flag[0]): playing_flag[1]}) # make sure that the other robot is not playing

    robot.set_output(playing_flag[0], playing_flag[2]) # signal that this robot is making a move

    row, col = chip_postion(board) # randomly select a random open position on the board

    flag = position_to_pins[(row, col)] # get corresponding pins

    board[row][col] = "X" if move_number % 2 == 0 else "O"  # update board matrix

    # executes the robots motion
    robot.pick_n_place(
        pick_pose=chip_pick_position[pick_index], # pose from which the robot pick the chip
        middle_pose=middle_pose, # pose between pick and place
        place_pose=chip_place_position[row, col], # pose where the robot places the chip
        end_pose=safe_pose, # clear pose move to after placing the chip
        tcp=tcp_frame, # tool length and position (tool center point)
        speed=speed # speed of movement
        output=tcp_output
    )
    
    set_pins(PIN_MOVE_POSITIONS, move_position_pins) # set the move position using io pins

    set_pins([PIN_MOVE_IN_PROGRESS], [0]) # signal move completion


    robot.sleep(SLEEP_PERIOD) #  sleep to allow the other robot to read the move

    ########################
    # update the grab position index
    pick_index += 1

    # check for a win condition
    if check_win(board, player):
        print(f"Player {player} wins!")
        # move to safe position and exit loop
        robot.jmove(safe_pose)
        break # exit loop if the player wins
else:
        # no open position found, end the game
        print("It's a draw!")
        robot.jmove(safe_pose)  # move to safe position
        break # exit loop if there is a draw

    # signal that this robot has completed the move
    set_pins([PIN_MOVE_IN_PROGRESS], [0])

    # wait for the other robot's turn to complete
    while read_pins([PIN_MOVE_IN_PROGRESS])[0] == 1:
        robot.sleep(SLEEP_PERIOD)  # waiting interval

# Close connection
robot.close()